<a href="https://colab.research.google.com/github/sonarosa/zero_shot_image/blob/main/Zero_shot_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
# More models in the model hub.
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
image_to_classify = "/content/image-classification-input.jpeg"
labels_for_classification =  ["cat and dog",
                              "lion and cheetah",
                              "rabbit and lion"]
scores = classifier(image_to_classify,
                    candidate_labels = labels_for_classification)


In [ ]:
[{'score': 0.9950482249259949, 'label': 'cat and dog'},
{'score': 0.004863627254962921, 'label': 'rabbit and lion'},
{'score': 8.816882473183796e-05, 'label': 'lion and cheetah'}]


[{'score': 0.9950482249259949, 'label': 'cat and dog'},
 {'score': 0.004863627254962921, 'label': 'rabbit and lion'},
 {'score': 8.816882473183796e-05, 'label': 'lion and cheetah'}]

In [ ]:
print(f"The highest score is {scores[0]['score']:.3f} for the label {scores[0]['label']}")


The highest score is 0.878 for the label cat and dog


In [2]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

def load_image(image_path):
    """
    Load and preprocess the image.
    """
    preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
    image = Image.open(image_path)
    inputs = preprocess(images=image, return_tensors="pt")
    return inputs

def load_clip_model():
    """
    Load the CLIP model.
    """
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
    return model

def calculate_feature_vector(image, model):
    """
    Calculate the feature vector of the image using the CLIP model.
    """
    with torch.no_grad():
        feature_vector = model.get_image_features(pixel_values=image.pixel_values)
    return feature_vector

def cosine_similarity(vector1, vector2):
    """
    Calculate the cosine similarity between two vectors.
    """
    # Ensure both vectors have shape (1, D) or (N, D)
    if len(vector1.shape) == 1:
        vector1 = vector1.unsqueeze(0)
    if len(vector2.shape) == 1:
        vector2 = vector2.unsqueeze(0)

    # Calculate cosine similarity
    similarity = torch.nn.functional.cosine_similarity(vector1, vector2, dim=1)
    return similarity.item()


def detect_anomalies(image_path, threshold=0.5):
    """
    Detect anomalies in an image using cosine similarity between the image's feature vector and a reference vector.
    """
    reference_vector = torch.randn(512)  # Example reference vector, you can replace it with any vector

    inputs = load_image(image_path)
    model = load_clip_model()

    image_vector = calculate_feature_vector(inputs, model)
    similarity = cosine_similarity(image_vector, reference_vector)

    return similarity < threshold  # Anomaly detected if similarity is less than the threshold

# Example usage
if __name__ == "__main__":
    image_path = "/content/maxresdefault.jpg"
    anomaly_detected = detect_anomalies(image_path)
    if anomaly_detected:
        print("Anomaly detected in the image!")
    else:
        print("No anomaly detected.")


config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Anomaly detected in the image!
